In [1]:
# 파일 불러오기
# 파일 링크 : https://dacon.io/competitions/official/235871/data
import pandas as pd
X_train_total=pd.read_csv("train.csv")
X_test=pd.read_csv("test.csv")

In [2]:
# X_train/y_train 데이터 설정
X_train=X_train_total.drop("target", axis=1)
y_train=X_train_total[["target"]]

In [3]:
# print(X_train.info())
# 데이터 
# Gender, Customer Type, Type of Travel, Class 바꿔야함
# train과 test에 같은 속성이 있는지 갯수 확인(?)
a=X_train['Gender'].unique()
print(len(a))
b=X_test['Gender'].unique()
print(len(b))

2
2


In [4]:
a=X_train['Customer Type'].unique()
print(len(a))
b=X_test['Customer Type'].unique()
print(len(b))

2
2


In [5]:
a=X_train['Type of Travel'].unique()
print(len(a))
b=X_test['Type of Travel'].unique()
print(len(b))

2
2


In [6]:
a=X_train['Class'].unique()
print(len(a))
b=X_test['Class'].unique()
print(len(b))

3
3


In [7]:
# cust_id만 따로 저장..!
cust_id=X_test[["id"]]

In [8]:
# id컬럼은 필요 없으니 삭제
X_train=X_train.drop('id', axis=1)
X_test=X_test.drop('id', axis=1)
# 더미변환
X_train_dum=pd.get_dummies(X_train)
X_test_dum=pd.get_dummies(X_test)

In [9]:
# 더미변환 후 데이터 형태 확인
print(X_train_dum.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 27 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Age                                3000 non-null   int64  
 1   Flight Distance                    3000 non-null   int64  
 2   Seat comfort                       3000 non-null   int64  
 3   Departure/Arrival time convenient  3000 non-null   int64  
 4   Food and drink                     3000 non-null   int64  
 5   Gate location                      3000 non-null   int64  
 6   Inflight wifi service              3000 non-null   int64  
 7   Inflight entertainment             3000 non-null   int64  
 8   Online support                     3000 non-null   int64  
 9   Ease of Online booking             3000 non-null   int64  
 10  On-board service                   3000 non-null   int64  
 11  Leg room service                   3000 non-null   int64

In [10]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
minmax=MinMaxScaler()
minmax.fit(X_train_dum)
minmax_X_train=minmax.transform(X_train_dum)
minmax_X_test=minmax.transform(X_test_dum)

In [11]:
# minmax 정규화가 잘 적용되었는지 확인
print(pd.DataFrame(minmax_X_train).describe())

                0            1            2            3            4   \
count  3000.000000  3000.000000  3000.000000  3000.000000  3000.000000   
mean      0.441137     0.282735     0.572667     0.601933     0.574867   
std       0.206970     0.150528     0.278996     0.303909     0.286302   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.273973     0.189788     0.400000     0.400000     0.400000   
50%       0.438356     0.275988     0.600000     0.600000     0.600000   
75%       0.602740     0.365337     0.800000     0.800000     0.800000   
max       1.000000     1.000000     1.000000     1.000000     1.000000   

                5            6            7           8            9   ...  \
count  3000.000000  3000.000000  3000.000000  3000.00000  3000.000000  ...   
mean      0.504167     0.651933     0.670467     0.62625     0.697600  ...   
std       0.323678     0.264537     0.270565     0.32767     0.260442  ...   
min       0.000000   

In [12]:
# 랜덤포레스트 모델 생성
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(random_state=42)

In [13]:
# 모델 적용, train target 예측
model.fit(minmax_X_train, y_train.values.ravel())
prob_train=model.predict(minmax_X_train)
print(model.score(minmax_X_train, y_train.values.ravel()))

1.0


In [14]:
# test target 예측
prob_test=model.predict(minmax_X_test)
prob_test=pd.DataFrame(prob_test)

In [15]:
# cust_id 컬럼과 예측 컬럼 병합
total=pd.concat([cust_id, prob_test], axis=1)

# 컬럼명 변경
total.columns=['id', 'target']
print(total)

        id  target
0        1       1
1        2       0
2        3       1
3        4       1
4        5       1
...    ...     ...
1995  1996       0
1996  1997       1
1997  1998       0
1998  1999       1
1999  2000       1

[2000 rows x 2 columns]


In [16]:
# 파일 내보내기
total.to_csv('cust_predcit.csv', index=False)